# Modelado de Series Temporales de Aparcamientos usando RNNs y optimización con Optuna
En este notebook se abordará un problema de predicción de series temporales utilizando datos de disponibilidad de parkings.

Flujo de trabajo:
1. Agrupación de los datos por id de aparcamiento (idAparcamiento), para tratar cada parking como una serie temporal independiente.
2. División del conjunto de datos en tres subconjuntos: entrenamiento (train), validación (val) y prueba (test).
3. Entrenamiento de modelos de redes neuronales recurrentes simples (vanilla):
 - Vanilla RNN
 - Vanilla GRU
 - Vanilla LSTM
4. Ajuste de hiperparámetros utilizando Optuna para encontrar la configuración óptima en cada tipo de modelo.
5. Comparación de los resultados de rendimiento entre los distintos modelos utilizando métricas apropiadas.

El objetivo final es evaluar qué arquitectura ofrece mejores resultados para este tipo de datos y tarea de predicción.


In [87]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import torchmetrics
from torchmetrics import MetricCollection
from torchmetrics.regression import MeanAbsoluteError, MeanSquaredError, MeanAbsolutePercentageError, R2Score

from tqdm import tqdm 
import time

import optuna


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
HIDDEN_SIZE = 32
NUM_LAYERS = 1
LEARNING_RATE = 1e-3

## 1. Cargar los datos

In [3]:
df = pd.read_csv("../data/processed/data_processed.csv")

#convertir a indice
df.set_index("timestamp", inplace= True)
df.index = pd.to_datetime(df.index)

df

,idAparcamiento,PlazasTotales,PlazasDisponibles,PorcPlazasDisponibles,year,month,day,weekday
timestamp,,,,,,,,
2023-02-03 10:00:00,6,372.0,60.0,16.129032,2023,2,3,4
2023-02-03 11:00:00,6,372.0,48.0,12.903226,2023,2,3,4
2023-02-03 12:00:00,6,372.0,66.0,17.741935,2023,2,3,4
2023-02-03 13:00:00,6,372.0,119.0,31.989247,2023,2,3,4
2023-02-03 14:00:00,6,372.0,155.0,41.666667,2023,2,3,4
...,...,...,...,...,...,...,...,...
2025-03-05 03:00:00,78,464.0,355.0,76.508621,2025,3,5,2
2025-03-05 04:00:00,78,464.0,355.0,76.508621,2025,3,5,2
2025-03-05 05:00:00,78,464.0,356.0,76.724138,2025,3,5,2


## 2. División del conjunto de datos

Realizamos la división del conjunto de datos, agrupando por `idAparcamiento`. 
La idea es que el conjunto de **test sea común** para todos los aparcamientos, correspondiente al **último 10% del rango temporal total** del dataset. 

El resto de los datos disponibles para cada parking se dividen en:

- **Entrenamiento (train)**: el primer 85% de los datos previos al test.
- **Validación (val)**: el último 15% de los datos previos al test.

In [4]:
from datetime import timedelta

# 1. Calcular el rango temporal global
fecha_min_global = df.index.min()
fecha_max_global = df.index.max()
rango_total = fecha_max_global - fecha_min_global

# 2. Calcular el inicio del conjunto de test (último 10% del rango)
test_ratio = 0.10
test_duration = timedelta(seconds=rango_total.total_seconds() * test_ratio)
test_start = fecha_max_global - test_duration

# 3. Diccionarios para almacenar los splits
train_dict = {}
val_dict = {}
test_dict = {}

val_ratio = 0.1  # del conjunto anterior al test

# 4. División por parking
for parking_id, group in df.groupby("idAparcamiento"):
    group = group.sort_index()
    
    # Split basado en el corte global
    test_set = group[group.index >= test_start]
    remaining = group[group.index < test_start]

    # Dividir en train y val
    val_size = int(len(remaining) * val_ratio)
    val_set = remaining.iloc[-val_size:]
    train_set = remaining.iloc[:-val_size]
    
    # Guardar resultados
    train_dict[parking_id] = train_set
    val_dict[parking_id] = val_set
    test_dict[parking_id] = test_set




## 3. Definir `Dataset` de pytorch

In [5]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, input_window, output_window, feature_cols, target_col):
        """
        data: DataFrame que contiene los datos de la serie temporal
        input_window: nº de pasos de tiempo en la secuencia de entrada
        output_window: nº de pasos de tiempo a predecir
        feature_cols: lista de nombres de columnas que se usan como característcas
        target_col: nombre de la variable a predecir
        """
        self.data = data
        self.input_window = input_window
        self.output_window = output_window
        self.feature_cols = feature_cols
        self.target_cols = target_col

    def __len__(self):
        """
        Función que devuele el nº de datos del Dataset
        """
        return len(self.data) - self.input_window - self.output_window + 1 #

    def __getitem__(self, idx):
        """
        Función que devuelve un dato a partir de un índice
        """
        X = self.data[idx: idx + self.input_window][self.feature_cols].values
        Y = self.data[idx + self.input_window: idx + self.input_window + self.output_window][self.target_cols].values
        
        X_tensor = torch.tensor(X, dtype= torch.float32)
        Y_tensor = torch.tensor(Y, dtype= torch.float32)

        return X_tensor, Y_tensor

In [6]:
feature_cols = ['PorcPlazasDisponibles']  
target_col = 'PorcPlazasDisponibles'     
input_window = 24         # Número de pasos de tiempo en la secuencia de entrada
output_window = 1         # Número de pasos de tiempo a predecir

Dado que tenemos una serie temporal por parking, construimos un diccionario, donde las claves son los identificadores de parkings y los valores son Daatasets de tipo `torch.utils.data.Dataset`.

In [7]:
train_datasets = {
    pid: TimeSeriesDataset(df, input_window, output_window, feature_cols, target_col=target_col)
    for pid, df in train_dict.items()
}
val_datasets = {
    pid: TimeSeriesDataset(df, input_window, output_window, feature_cols, target_col=target_col)
    for pid, df in val_dict.items()
}
test_datasets = {
    pid: TimeSeriesDataset(df, input_window, output_window, feature_cols, target_col=target_col)
    for pid, df in test_dict.items()
}

In [8]:
for pid, df in train_datasets.items():
    print("id parking: ", pid)
    print("longitud: ", len(df))

id parking:  6
longitud:  13908
id parking:  7
longitud:  13908
id parking:  8
longitud:  22507
id parking:  13
longitud:  13908
id parking:  34
longitud:  9310
id parking:  75
longitud:  22505
id parking:  77
longitud:  10735
id parking:  78
longitud:  7454


## 4. Crear `DataLoaders` a partir de `Dataset`

Para crear los `DataLoader`, seguimos el mismo criterio, es decir, crear un diccionario de DataLoaders, en el que cada iteración nos devuelve un batch de datos para cada parking.
- Cada batch de datos debe tener dimensión: `(batch_size, window_size, n_features)`

In [9]:
train_dataloaders = {
    pid: DataLoader(ts_dataset, batch_size = 32, shuffle = True) #(batch_size, window_size, n_features)
    for pid, ts_dataset in train_datasets.items()
}

val_dataloaders = {
    pid: DataLoader(ts_dataset, batch_size = 32, shuffle = True) #(batch_size, window_size, n_features)
    for pid, ts_dataset in val_datasets.items()
}

test_dataloaders = {
    pid: DataLoader(ts_dataset, batch_size = 64) #(batch_size, window_size, n_features)
    for pid, ts_dataset in test_datasets.items()
}

In [10]:
for pid, dloader in train_dataloaders.items():
    print("id parking", pid)
    for batch in dloader:
        print("Dimensión del primer batch de datos:", batch[0].shape)
        print("Dimensión del primer batch de etiquetas: ", batch[1].shape)
        break

    print("\n")

id parking 6
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 7
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 8
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 13
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 34
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 75
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 77
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 78
Dimensión del pr

## 5. Definir Modelos (`RNN`,`GRU`,`LSTM`)


In [11]:
class VanillaRNN(nn.Module):
    """
    Clase que implementa una RNN vanilla: 1 capa, 1 neurona por capa oculta
    """
    def __init__(self, input_size, hidden_size, output_size, num_layers = 1):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size = self.input_size,
                          hidden_size = self.hidden_size,
                          num_layers = self.num_layers,
                          batch_first = True)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, x, h0 = None):
        """ 
        
        """
        batch_size_ = x.size(0)
        #si no se pasa el primer estado oculto, lo inicializamos con 0
        if h0 is None:
            h0 = torch.zeros(self.num_layers, batch_size_, self.hidden_size).to(x.device)
        
        #all_hidden_states (output) --> todos los estados ocultos (batch_size, seq_length, hidden_size)
        #last_hidden_state (h_n) --> ultimo estado oculto (num_layers, batch_size, hidden_size)
        all_hidden_states, last_hidden_state = self.rnn(x, h0)

        last_hidde_state_of_last_layer = last_hidden_state[-1]
        
        #si no pasamos por una capa densa, el modelo no hace la prediccion
        pred = self.fc(last_hidde_state_of_last_layer)  # (batch_size, output_size)

        return pred

Comprobamos que no hay inconsistencias

In [12]:
model = VanillaRNN(input_size=1, hidden_size=64, output_size=1, num_layers=1)
x = torch.randn(32, 24, 1)  # (batch_size, sequence_length, num_features)
y  = model(x)
print(y.shape)  # → (32, 1)


torch.Size([32, 1])


In [ ]:
import torch
import torch.nn as nn

class VanillaLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            batch_first=True
        )
        
        self.fc = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, h0=None, c0=None):
        batch_size_ = x.size(0)
        #si no se pasa el primer h0 o c0, lo inicializamos con 0
        if h0 is None:
            h0 = torch.zeros(self.num_layers, batch_size_, self.hidden_size).to(x.device)
        if c0 is None:
            c0 = torch.zeros(self.num_layers, batch_size_, self.hidden_size).to(x.device)

        all_hidden_states, (last_hidden_state, last_cell_memory) = self.lstm(x, (h0, c0))

        # Usar el último estado oculto de la última capa
        pred = self.fc(last_hidden_state[-1])  # Shape: (batch_size, output_size)
        
        return pred


Comprobamos que no hay inconsistencias

In [14]:
model = VanillaLSTM(input_size=1, hidden_size=64, output_size=1, num_layers=1)
x = torch.randn(32, 24, 1)  # (batch_size, sequence_length, num_features)
y  = model(x)
print(y.shape)  #(32, 1)

torch.Size([32, 1])


In [16]:
class VanillaGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers = 1):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.gru = nn.GRU(input_size = self.input_size,
                          hidden_size = self.hidden_size,
                          num_layers = self.num_layers,
                          batch_first= True)
        
        self.fc = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, x, h0=None):
        
        batch_size_ = x.size(0)
        
        ##si no se pasa el primer h0, lo inicializamos con 0
        if h0 is None:
            h0 = torch.zeros(self.num_layers, batch_size_, self.hidden_size).to(x.device)
        
        #all_hidden_states (output) --> todos los estados ocultos (batch_size, seq_length, hidden_size)
        #last_hidden_state (h_n) --> ultimo estado oculto (num_layers, batch_size, hidden_size)
        all_hidden_states, last_hidden_state = self.gru(x, h0)

        last_hidde_state_of_last_layer = last_hidden_state[-1]
        
        #si no pasamos por una capa densa, el modelo no hace la prediccion
        pred = self.fc(last_hidde_state_of_last_layer)  # (batch_size, output_size)

        return pred  



In [17]:
model = VanillaGRU(input_size=1, hidden_size=64, output_size=1, num_layers=1)
x = torch.randn(32, 24, 1)  # (batch_size, sequence_length, num_features)
y  = model(x)
print(y.shape)  #(32, 1)

torch.Size([32, 1])


## 6. Definir métricas

In [23]:
metrics = MetricCollection({
    "MAE": MeanAbsoluteError(),
    "MSE": MeanSquaredError(),
    "RMSE": MeanSquaredError(squared = False),
    "MAPE": MeanAbsolutePercentageError()
})


## 7. Instanciar modelos, optimizador, función de coste y learning rate scheduler

#### 💡 Estructura de entrenamiento modular por parking y arquitectura

Dado que tenemos:

- Un diccionario de datasets por `idAparcamiento`
- Tres arquitecturas distintas: `RNN`, `LSTM`, y `GRU`

Cada modelo se entrena **por separado para cada parking**, permitiendo una evaluación más precisa por serie temporal individual.

Para hacerlo **flexible y escalable**, organizamos todos los componentes en **diccionarios anidados**. Cada uno almacena los elementos necesarios para entrenar y validar por `idAparcamiento` y por tipo de modelo:

- `models["rnn"][pid]`, `models["lstm"][pid]`, etc.
- `optimizers["gru"][pid]`, `criterions["lstm"][pid]`, etc.
- `schedulers["rnn"][pid]` para aplicar estrategias de LR por modelo

Esto nos permite:

- Entrenar múltiples arquitecturas de forma aislada y comparable  
- Hacer tuning o validación cruzada por parking  
- Comparar vanilla vs. modelos optimizados con Optuna  
- Guardar y cargar modelos individuales por ID

Esta estructura es especialmente útil en contextos donde las series tienen longitudes o comportamientos distintos, como ocurre con los parkings reales.


In [20]:
def get_input_output_size(dataloader):
    """
    Extrae el número de features de entrada y el tamaño de salida
    desde un batch del DataLoader.
    
    Retorna:
        input_size: int (número de features por paso temporal)
        output_size: int (dimensión de la predicción por muestra)
    """
    # Tomamos el primer batch del dataloader
    batch = next(iter(dataloader))
    x, y = batch
    input_size = x.shape[-1]   # última dimensión de x --> n_features
    output_size = y.shape[-1]  # última dimensión de y --> n_outputs
    return input_size, output_size


In [ ]:
dloader = train_dataloaders[6]
INPUT_SIZE, OUTPUT_SIZE = get_input_output_size(dloader)

print("Input size (n_features):", INPUT_SIZE)
print("Output size:", OUTPUT_SIZE)

Input size (n_features): 1
Output size: 1


1. Definir estructuras adecuadas: diccionarios de diccionarios

In [65]:
models = {
    "rnn": {},
    "lstm": {},
    "gru": {}
}

criterions = {
    "rnn": {},
    "lstm": {},
    "gru": {}
}

optimizers = {
    "rnn": {},
    "lstm": {},
    "gru": {}
}

schedulers = {
    "rnn": {},
    "lstm": {},
    "gru": {}
}

2. Definir una función para inicializar los componentes

In [66]:
def init_model_components(model_class, input_size, hidden_size, output_size, device, num_layers = 1, lr = 1e-3):
    """
    Función para instanciar los componentes: modelo, función de pérdida, optimizer, learning rate scheduler
    """
    model = model_class(input_size, hidden_size, output_size, num_layers).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr = lr)

    scheduler = StepLR(
    optimizer,   # tu optimizador
    step_size=10,  # reduce cada 10 epochs
    gamma=0.05     # reduce a la mitad (lr = lr * gamma)
)

    return model, criterion, optimizer, scheduler


3. Construir modelos, criterions, optimizers y schedulers por parking

In [67]:
HIDDEN_SIZE = 32
NUM_LAYERS = 1
LEARNING_RATE = 1e-3

In [72]:
DEVICE

'cuda'

In [73]:
for pid, dataset in train_datasets.items():

    #nº de caracteristicas para predecir de salida = 1 (disponibilidad en instantes anteriores)
    INPUT_SIZE = dataset[0][0].shape[-1]

    #tamaño de salida = 1 (predecir valor siguiente a partir de los 24 anteriores)
    OUTPUT_SIZE = dataset[0][1].shape[-1]

    #RNN
    model, criterion, optimizer, scheduler = init_model_components(VanillaLSTM, 
                                                                   input_size = INPUT_SIZE, 
                                                                   hidden_size = HIDDEN_SIZE,
                                                                   output_size = OUTPUT_SIZE,
                                                                   num_layers = NUM_LAYERS,
                                                                   device = DEVICE,
                                                                   lr= LEARNING_RATE)
    models["rnn"][pid] = model
    criterions["rnn"][pid] = criterion
    optimizers["rnn"][pid] = optimizer
    schedulers["rnn"][pid] = scheduler

    #LSTM
    model, criterion, optimizer, scheduler = init_model_components(VanillaLSTM,
                                                                   input_size= INPUT_SIZE,
                                                                   hidden_size = HIDDEN_SIZE,
                                                                   output_size = OUTPUT_SIZE,
                                                                   num_layers = NUM_LAYERS,
                                                                   device = DEVICE,
                                                                   lr= LEARNING_RATE)
    models["lstm"][pid] = model
    criterions["lstm"][pid] = criterion
    optimizers["lstm"][pid] = optimizer
    schedulers["lstm"][pid] = scheduler

    #GRU
    model, criterion, optimizer, scheduler = init_model_components(VanillaGRU,
                                                                   input_size= INPUT_SIZE,
                                                                   hidden_size = HIDDEN_SIZE,
                                                                   output_size = OUTPUT_SIZE,
                                                                   num_layers = NUM_LAYERS,
                                                                   device = DEVICE,
                                                                   lr= LEARNING_RATE)
    models["gru"][pid] = model
    criterions["gru"][pid] = criterion
    optimizers["gru"][pid] = optimizer
    schedulers["gru"][pid] = scheduler

In [74]:
models

{'rnn': {6: VanillaLSTM(
    (lstm): LSTM(1, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=1, bias=True)
  ),
  7: VanillaLSTM(
    (lstm): LSTM(1, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=1, bias=True)
  ),
  8: VanillaLSTM(
    (lstm): LSTM(1, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=1, bias=True)
  ),
  13: VanillaLSTM(
    (lstm): LSTM(1, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=1, bias=True)
  ),
  34: VanillaLSTM(
    (lstm): LSTM(1, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=1, bias=True)
  ),
  75: VanillaLSTM(
    (lstm): LSTM(1, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=1, bias=True)
  ),
  77: VanillaLSTM(
    (lstm): LSTM(1, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=1, bias=True)
  ),
  78: VanillaLSTM(
    (lstm): LSTM(1, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=1, bias

## 8. Crear callbacks

In [75]:
class EarlyStopping:
    """
    Callback para detener el entrenamiento anticipadamente si la métrica de validación
    no mejora tras un número de épocas determinado (patience).

    Args:
        patience (int): Número de épocas sin mejora antes de detener.
        min_delta (float): Cambio mínimo considerado como mejora.
    """
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.should_stop = False

    def __call__(self, val_loss):
        """
        Evalúa si se debe detener el entrenamiento.

        Args:
            val_loss (float): Pérdida (loss) de validación actual.

        Returns:
            bool: True si debe detenerse, False en caso contrario.
        """
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
        return self.should_stop

In [76]:
def save_best_model(model, val_loss, best_loss, path):
    """
    Guarda el modelo si su loss de validación mejora respecto al anterior.

    Args:
        model (nn.Module): Modelo actual.
        val_loss (float): Loss de validación actual.
        best_loss (float): Mejor loss registrado hasta el momento.
        path (str): Ruta donde guardar el modelo si mejora.

    Returns:
        float: Nuevo mejor loss (actual o anterior si no mejoró).
    """
    if val_loss < best_loss:
        torch.save(model.state_dict(), path)
        return val_loss  # se actualiza el best_loss
    return best_loss  # se mantiene igual


## 9. Definir función de validación

In [77]:
def validate_one_epoch(
        model,
        dataloader,
        criterion,
        metrics, 
        device 
):
    """
    Ejecuta una pasada de validación completa (1 epoch) sobre un dataloader.

    Args:
        model (nn.Module): Modelo PyTorch a evaluar.
        dataloader (DataLoader): Dataloader con los datos de validación.
        criterion (nn.Module): Función de pérdida (loss).
        metrics (torchmetrics.MetricCollection): Métricas a evaluar.
        device (str): "cpu" o "cuda".

    Returns:
        dict: Diccionario con loss promedio y métricas calculadas.
    """

    model.eval() #poner modelo en modo evaluación

    #inicializar loss, nº batches y métricas
    running_loss = 0.0
    num_batches = 0

    metrics.to(device)
    metrics.reset()

    with torch.no_grad(): #desactivar cálculo de gradientes
        for x_batch, y_batch in dataloader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            # 1. Forward
            y_pred = model(x_batch)

            # 2. Calcular loss y añadirlo a lista de losses
            loss = criterion(y_pred, y_batch)
            running_loss += loss.item()
            num_batches += 1

            #acumular valores del batch
            metrics.update(y_pred, y_batch)
    
    #calcular loss promedio del batch
    avg_loss = running_loss/num_batches

    #calcular metrica total: promedio de metricas de cada batch
    metric_results = metrics.compute()

    #crear diccionario de resutaldos: primer elemento del diccionario es el loss
    results = {"val_loss": avg_loss}

    #añadir elementos al diccionario: cada par clave-valor es una métrica
    results.update({k:v.item() for k,v in metric_results.items()})

    return results

In [103]:
from tqdm import tqdm
import time

def validate_one_epoch(
    model,
    dataloader,
    criterion,
    metrics, 
    device,
    epoch=None
):
    """
    Ejecuta una pasada de validación completa (1 epoch) sobre un dataloader.

    Args:
        model (nn.Module): Modelo PyTorch a evaluar.
        dataloader (DataLoader): Dataloader con los datos de validación.
        criterion (nn.Module): Función de pérdida (loss).
        metrics (torchmetrics.MetricCollection): Métricas a evaluar.
        device (str): "cpu" o "cuda".
        epoch (int, optional): Número de epoch actual (solo para mostrar).

    Returns:
        dict: Diccionario con loss promedio y métricas calculadas.
    """

    model.eval()  # poner modelo en modo evaluación

    running_loss = 0.0
    num_batches = len(dataloader)

    metrics.to(device)
    metrics.reset()

    start_time = time.time()

    # tqdm para barra de progreso
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch} [Val]", leave=False)

    with torch.no_grad():
        for x_batch, y_batch in progress_bar:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            # Forward
            y_pred = model(x_batch)

            # Calcular loss y acumularlo
            loss = criterion(y_pred, y_batch)
            running_loss += loss.item()

            # Actualizar métricas
            metrics.update(y_pred, y_batch)

            # Mostrar loss promedio hasta el batch actual
            batch_idx = progress_bar.n
            avg_loss_so_far = running_loss / (batch_idx + 1)
            progress_bar.set_postfix({"Loss": avg_loss_so_far})

    #calcular loss promedio del batch
    avg_loss = running_loss / num_batches

    #calcular metrica total: promedio de metricas de cada batch
    metric_results = metrics.compute()

    #crear diccionario de resutaldos: primer elemento del diccionario es el loss
    results = {"val_loss": avg_loss}
    #añadir elementos al diccionario: cada par clave-valor es una métrica
    results.update({k: v.item() for k, v in metric_results.items()})

    elapsed_time = time.time() - start_time
    print(f"Epoch {epoch} - Val   | Loss: {avg_loss:.4f} | Time: {elapsed_time:.1f}s")

    return results


Probamos que no hay inconsistencias, a pesar de que los pesos de los modelos sean aleatorios

In [104]:
val_results = validate_one_epoch(
    model=models["lstm"][6],
    dataloader=val_dataloaders[6],
    criterion=criterions["lstm"][6],
    metrics = metrics,
    device= DEVICE,
    epoch=1
)

print(val_results)


Epoch 1 - Val   | Loss: 81.9369 | Time: 0.6s
{'val_loss': 81.93691555658977, 'MAE': 7.372611999511719, 'MAPE': 28434.85546875, 'MSE': 82.05606079101562, 'RMSE': 9.058480262756348}


## 10. Definir bucle de entrenamiento

In [96]:
def train_one_epoch(
                model,
                dataloader,
                criterion,
                optimizer,
                metrics,
                device,
                epoch = None
):
        """
        Ejecuta una epoch completa de entrenamiento.

        Args:
                model (nn.Module): Modelo a entrenar.
                dataloader (DataLoader): Dataloader con datos de entrenamiento.
                criterion (nn.Module): Función de pérdida.
                optimizer (Optimizer): Optimizador para actualizar pesos.
                metrics (torchmetrics.MetricCollection): Métricas de evaluación.
                device (str): "cpu" o "cuda".
                epoch (int, optional): Número de la época actual (solo para mostrar).

        Returns:
                dict: Diccionario con loss promedio y métricas acumuladas.
        """
        
        model.train() #poner modelo en modo entrenamiento

        #inicializar loss y nº de batches
        running_loss = 0.0
        num_batches = len(dataloader)

        #inicializar métricas y moverlas al device
        metrics.to(device)
        metrics.reset()

        #coger tiempo actual de referencia
        start_time = time.time()

        #inicializar barra de progreso
        #progress_bar envuelve el dataloader y lo monitoriza
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}", leave=False)


        for x_batch, y_batch in progress_bar:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                optimizer.zero_grad() #1. resetear gradientes

                y_pred = model(x_batch) #2. Forward

                loss = criterion(y_pred, y_batch) # 3. calcular loss
                running_loss += loss.item()
                
                batch_idx = progress_bar.n #indice del batch
                avg_loss_so_far = running_loss / (batch_idx + 1)

                #Mostrar en tiempo real el loss
                progress_bar.set_postfix({"Loss": avg_loss_so_far})


                loss.backward() #4. Propagar loss
                optimizer.step() #5. Actualizar pesos


                metrics.update(y_pred, y_batch)
        
        #calcular loss promedio del batch
        avg_loss = running_loss/num_batches

        #calcular metrica total: promedio de metricas de cada batch
        metric_results = metrics.compute()

        #crear diccionario de resutaldos: primer elemento del diccionario es el loss
        results = {"train_loss": avg_loss}

        #añadir elementos al diccionario: cada par clave-valor es una métrica
        results.update({k: v.item() for k,v in metric_results.items()})

        elapsed_time = time.time() - start_time
        print(f"Epoch {epoch} - Train | Loss: {avg_loss:.4f} | Time: {elapsed_time:.1f}s")

        return results

Probamos que no hay inconsistencias, a pesar de que los pesos de los modelos sean aleatorios

In [95]:
train_results = train_one_epoch(
    model=models["lstm"][6],
    dataloader=train_dataloaders[6],
    criterion=criterions["lstm"][6],
    optimizer=optimizers["lstm"][6],
    metrics=metrics,
    device=DEVICE,
    epoch = 1
)

print(train_results)


Epoch 1 - Train | Loss: 93.8941 | Time: 6.1s
{'train_loss': 93.89411600924086, 'MAE': 7.749772548675537, 'MAPE': 3310.0859375, 'MSE': 93.90457153320312, 'RMSE': 9.690437316894531}


## 11. Optimizació óptima de hiperparámetros con `Optuna`

## 12. Entrenamiento

In [107]:
def fit(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    metrics,
    scheduler=None,
    num_epochs=10,
    device="cpu"
):
    """
    Entrena un modelo durante varias épocas y evalúa en validación.

    Args:
        model (nn.Module): Modelo a entrenar.
        train_loader (DataLoader): Dataloader de entrenamiento.
        val_loader (DataLoader): Dataloader de validación.
        criterion (nn.Module): Función de pérdida.
        optimizer (Optimizer): Optimizador.
        metrics (torchmetrics.MetricCollection): Métricas compartidas entre train/val.
        scheduler (optional): Scheduler de LR.
        num_epochs (int): Número de épocas.
        device (str): "cpu" o "cuda".

    Returns:
        list: Lista de dicts con métricas de cada época.
    """

    history = []  # Para guardar métricas por época

    model.to(device)

    for epoch in range(1, num_epochs + 1):
        print(f"Epoch {epoch}/{num_epochs}")

        # Entrenamiento
        train_metrics = train_one_epoch(
            model=model,
            dataloader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            metrics=metrics.clone(),  # ← importante: evita conflictos
            device=device,
            epoch=epoch
        )

        # Validación
        val_metrics = validate_one_epoch(
            model=model,
            dataloader=val_loader,
            criterion=criterion,
            metrics=metrics.clone(),
            device=device,
            epoch=epoch
        )

        # Scheduler (opcional)
        scheduler.step()

        # Unir métricas
        combined = {"epoch": epoch}
        combined.update(train_metrics)
        combined.update(val_metrics)
        history.append(combined)

        # Mostrar resumen
        print(
            f"📊 Summary | Train Loss: {train_metrics['train_loss']:.4f} | "
            f"Val Loss: {val_metrics['val_loss']:.4f}"
        )

    return history


In [108]:
history = fit(
    model=models["lstm"][6],
    train_loader=train_dataloaders[6],
    val_loader=val_dataloaders[6],
    criterion=criterions["lstm"][6],
    optimizer=optimizers["lstm"][6],
    scheduler=schedulers["lstm"][6],
    metrics=metrics,  # una instancia de MetricCollection
    num_epochs=20,
    device=DEVICE
)


Epoch 1/20


Epoch 1 - Train | Loss: 14.9539 | Time: 6.0s


Epoch 1 - Val   | Loss: 16.3196 | Time: 0.8s
📊 Summary | Train Loss: 14.9539 | Val Loss: 16.3196
Epoch 2/20


Epoch 2 - Train | Loss: 14.9239 | Time: 6.1s


Epoch 2 - Val   | Loss: 16.4341 | Time: 0.6s
📊 Summary | Train Loss: 14.9239 | Val Loss: 16.4341
Epoch 3/20


...

Epoch 18 - Val   | Loss: 16.2183 | Time: 0.6s
📊 Summary | Train Loss: 14.8930 | Val Loss: 16.2183
Epoch 19/20


Epoch 19 - Train | Loss: 14.8904 | Time: 5.9s


Epoch 19 - Val   | Loss: 16.2984 | Time: 0.6s
📊 Summary | Train Loss: 14.8904 | Val Loss: 16.2984
Epoch 20/20


Epoch 20 - Train | Loss: 14.9233 | Time: 5.9s


Epoch 20 - Val   | Loss: 16.2800 | Time: 0.6s
📊 Summary | Train Loss: 14.9233 | Val Loss: 16.2800


## 13. Predicciones sobre el conjunto de test

In [109]:
def predict(model, dataloader, device="cpu"):
    """
    Obtiene las predicciones del modelo sobre un dataloader completo.

    Args:
        model (nn.Module): Modelo entrenado.
        dataloader (DataLoader): Dataloader a predecir (train, val o test).
        device (str): "cpu" o "cuda".

    Returns:
        Tuple[Tensor, Tensor]: (y_true, y_pred), ambos de tamaño (N, output_size)
    """
    model.eval()
    model.to(device)

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            y_pred = model(x_batch)

            all_preds.append(y_pred.cpu())
            all_targets.append(y_batch.cpu())

    y_pred = torch.cat(all_preds, dim=0)
    y_true = torch.cat(all_targets, dim=0)

    return y_true, y_pred


In [110]:
y_true, y_pred = predict(
    model=models["lstm"][6],
    dataloader=test_dataloaders[6],
    device=DEVICE
)

print(y_true.shape, y_pred.shape)


torch.Size([2758, 1]) torch.Size([2758, 1])


In [111]:
y_true[:5]

tensor([[62.9032],
        [59.1398],
        [59.4086],
        [59.4086],
        [59.1398]])

In [112]:
y_pred[:5]

tensor([[67.1369],
        [53.9262],
        [55.5845],
        [59.7052],
        [58.4262]])

## 14. Exportación de checkpoints y logs